### Here is How We Can Build Complete Knowledge Graph for Multiple Record Sheets From Same File, To see content of Your file run the details.py.

##  Install Dependencies

In [ ]:
!pip install pandas neo4j openpyxl

## Import Libraries

In [ ]:
import pandas as pd
import re
from neo4j import GraphDatabase
from google.colab import files


## Upload File At Run Time

In [ ]:
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

with open(file_name, "r") as f:
    content = f.read()

uri = re.search(r"NEO4J_URI=(.*)", content).group(1).strip()
user = re.search(r"NEO4J_USERNAME=(.*)", content).group(1).strip()
password = re.search(r"NEO4J_PASSWORD=(.*)", content).group(1).strip()

print("Credentials loaded")

## Set KG Manager Instance

In [ ]:
class KGManager:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def clear_database(self):
        with self.driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n")
        print("🧹 Previous KG cleared.")

    def create_constraints(self):
        with self.driver.session() as session:
            queries = [
                "CREATE CONSTRAINT account_code IF NOT EXISTS FOR (a:Account) REQUIRE a.code IS UNIQUE",
                "CREATE CONSTRAINT transaction_voucher IF NOT EXISTS FOR (t:Transaction) REQUIRE t.voucher_no IS UNIQUE",
                "CREATE CONSTRAINT amount_id IF NOT EXISTS FOR (amt:Amount) REQUIRE amt.id IS UNIQUE",
                "CREATE CONSTRAINT month_val IF NOT EXISTS FOR (m:Month) REQUIRE m.value IS UNIQUE",
                "CREATE CONSTRAINT supplier_name IF NOT EXISTS FOR (s:Supplier) REQUIRE s.name IS UNIQUE",
                "CREATE CONSTRAINT customer_name IF NOT EXISTS FOR (c:Customer) REQUIRE c.name IS UNIQUE",
                "CREATE CONSTRAINT sku_name IF NOT EXISTS FOR (sku:SKU) REQUIRE sku.name IS UNIQUE"
            ]
            for q in queries:
                session.run(q)
        print("Constraints created.")

## Import Functions Required To Get Record From Multiple Sheets

In [ ]:

    # Import functions 
    def import_gl_finance(self, df):
        query = """
        UNWIND $rows AS row
        MERGE (acc:Account {code: row.`Account Code`})
          SET acc.name = row.`Account Name`, acc.class = row.`Class Name`
        MERGE (t:Transaction {voucher_no: row.`Voucher No`})
          SET t.date = date(row.`Transaction Date`), t.description = row.Description
        MERGE (m:Month {value: row.Month})
        MERGE (t)-[:OCCURRED_IN]->(m)
        FOREACH (_ IN CASE WHEN row.`Debit Amount` > 0 THEN [1] ELSE [] END |
            MERGE (amt:Amount {id: row.`Voucher No` + "-D-" + row.`Account Code`})
              SET amt.type = "Debit", amt.value = row.`Debit Amount`
            MERGE (t)-[:HAS_DEBIT]->(amt)
            MERGE (amt)-[:TO]->(acc)
        )
        FOREACH (_ IN CASE WHEN row.`Credit Amount` > 0 THEN [1] ELSE [] END |
            MERGE (amt:Amount {id: row.`Voucher No` + "-C-" + row.`Account Code`})
              SET amt.type = "Credit", amt.value = row.`Credit Amount`
            MERGE (t)-[:HAS_CREDIT]->(amt)
            MERGE (amt)-[:FROM]->(acc)
        )
        """
        self._run_query(df, query)

    def import_grn(self, df):
        query = """
        UNWIND $rows AS row
        MERGE (grn:GRN {number: row.GrnNumber})
          SET grn.date = date(row.GrnDate)
        MERGE (s:Supplier {name: row.SupplierName})
        MERGE (sku:SKU {name: row.SkuName})
        MERGE (w:Warehouse {name: row.WarehouseName})
        MERGE (grn)-[:FROM_SUPPLIER]->(s)
        MERGE (grn)-[:CONTAINS]->(sku)
        MERGE (grn)-[:STORED_IN]->(w)
        SET sku.qty = row.GrnItemQty
        """
        self._run_query(df, query)

    def import_purchase(self, df):
        query = """
        UNWIND $rows AS row
        MERGE (p:Purchase {invoice: row.InvoiceNumber})
          SET p.date = date(row.InvoiceDate), p.amount = row.NetAmount
        MERGE (s:Supplier {name: row.SupplierName})
        MERGE (sku:SKU {name: row.SkuName})
        MERGE (p)-[:FROM_SUPPLIER]->(s)
        MERGE (p)-[:CONTAINS]->(sku)
        SET sku.price = row.POItemUnitPrice, sku.qty = row.GRNItemQty
        """
        self._run_query(df, query)

    def import_purchase_return(self, df):
        query = """
        UNWIND $rows AS row
        MERGE (pr:PurchaseReturn {number: row.ReturnNumber})
          SET pr.date = date(row.ReturnDate), pr.amount = row.amount
        MERGE (s:Supplier {name: row.SupplierName})
        MERGE (sku:SKU {name: row.SkuName})
        MERGE (pr)-[:RETURNED_TO]->(s)
        MERGE (pr)-[:RETURNED_ITEM]->(sku)
        SET sku.return_qty = row.ReturnQty, sku.po_qty = row.POQty
        """
        self._run_query(df, query)

    def import_sale(self, df):
        query = """
        UNWIND $rows AS row
        MERGE (s:Sale {invoice: row.InvoiceNumber})
          SET s.date = date(row.DeliveryDate), s.amount = row.TotalNetAmount
        MERGE (c:Customer {name: row.CustomerName})
        MERGE (sku:SKU {name: row.SkuName})
        MERGE (s)-[:TO_CUSTOMER]->(c)
        MERGE (s)-[:CONTAINS]->(sku)
        SET sku.cartons = row.NoOfCartons
        """
        self._run_query(df, query)

    def import_sale_return(self, df):
        query = """
        UNWIND $rows AS row
        MERGE (sr:SaleReturn {number: row.SRNumber})
          SET sr.date = date(row.ReturnDate), sr.amount = row.Amount
        MERGE (c:Customer {name: row.CustomerName})
        MERGE (sku:SKU {name: row.SkuName})
        MERGE (sr)-[:FROM_CUSTOMER]->(c)
        MERGE (sr)-[:RETURNED_ITEM]->(sku)
        SET sku.return_qty = row.ReturnQty, sku.cost_price = row.CostPrice
        """
        self._run_query(df, query)

    def _run_query(self, df, query, batch_size=300):
        df = df.copy()
        if "Transaction Date" in df.columns:
            df["Transaction Date"] = pd.to_datetime(df["Transaction Date"], errors="coerce").dt.strftime("%Y-%m-%d")
            df["Month"] = pd.to_datetime(df["Transaction Date"], errors="coerce").dt.to_period("M").astype(str)
        if "GrnDate" in df.columns:
            df["GrnDate"] = pd.to_datetime(df["GrnDate"], errors="coerce").dt.strftime("%Y-%m-%d")
        if "InvoiceDate" in df.columns:
            df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce").dt.strftime("%Y-%m-%d")
        if "ReturnDate" in df.columns:
            df["ReturnDate"] = pd.to_datetime(df["ReturnDate"], errors="coerce").dt.strftime("%Y-%m-%d")
        if "DeliveryDate" in df.columns:
            df["DeliveryDate"] = pd.to_datetime(df["DeliveryDate"], errors="coerce").dt.strftime("%Y-%m-%d")

        records = df.to_dict("records")
        with self.driver.session() as session:
            for start in range(0, len(records), batch_size):
                end = start + batch_size
                session.run(query, rows=records[start:end])
                print(f"Batch {start}-{end} imported.")

## Upload Data File, EXCEL File

In [ ]:
# Upload Excel

uploaded = files.upload()
excel_file = list(uploaded.keys())[0]
sheets = pd.read_excel(excel_file, sheet_name=None)

kg = KGManager(uri, user, password)
kg.clear_database()
kg.create_constraints()

print(f"Sheets found: {list(sheets.keys())}")

## These are sheets in the given excel file, You can change it accordingly with your own sheets names.

In [ ]:

# Import all sheets

for sheet_name, df in sheets.items():
    if sheet_name == "GL_Finance_Data":
        kg.import_gl_finance(df)
        print("Imported GL_Finance_Data ")
    elif sheet_name == "GRN":
        kg.import_grn(df)
        print("Imported GRN ")
    elif sheet_name == "Purchase":
        kg.import_purchase(df)
        print("Imported Purchase ")
    elif sheet_name == "Purchase Return":
        kg.import_purchase_return(df)
        print("Imported Purchase Return ")
    elif sheet_name == "Sale":
        kg.import_sale(df)
        print("Imported Sale ")
    elif sheet_name == "Sale Return":
        kg.import_sale_return(df)
        print("Imported Sale Return ")
    else:
        print(f"Skipping {sheet_name}, no mapping defined.")

kg.close()
print("Knowledge Graph import complete!")
